### DQN Docking Case

In [1]:
import sys

## !pip install -e ../gym_space_docking/

loc = os.popen('pip3 show gym_space_docking').readlines()[7].split()[1]


sys.path.append(loc)


import gym
import os
import gym_space_docking

In [2]:
# include Network
import tensorflow as tf
import numpy as np

### create enviroment pygame

In [3]:
env = gym.make('space_docking-v0')
print(env)

pygame 2.1.0 (SDL 2.0.16, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html
/Users/denny/Documents/workspace_ironhack/IronHack_Final_Project/gym_space_docking/gym_space_docking/envs
Discrete(7)
<OrderEnforcing<Space_Docking_Env<space_docking-v0>>>


In [4]:
env.reset()

In [5]:
print(env.action_space)
print(env.observation_space)
print(type(env.action_space))

Discrete(7)
Box([[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]], [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]], (800, 420), uint8)
<class 'gym.spaces.discrete.Discrete'>


In [6]:
# handle observations

# handle image scaling and resizing to fit the image in input data

def preprocess_observations(obs):
    pass

In [19]:
# create network
input_height = 88
input_width = 80
input_channels = 1
conv_n_maps = [32,64,64]
conv_kernel_sizes = [(8,8), (4,4), (3,3)]
conv_strides = [4, 2, 1]
conv_paddings = ['SAME'] * 3
conv_activation = [tf.nn.relu] * 3
n_hidden_in = 64 * 11 * 10 # 64 maps with size 11x10
n_hidden = 512
hidden_activation = tf.nn.relu
n_outputs = env.action_space.n# -> ToDo define in enviroment 
initializer =  tf.keras.initializers.VarianceScaling()# tf.contrib.layers.variance_scaling_initializer()

learning_rate = 0.01


In [24]:
tf.compat.v1.disable_eager_execution()

def q_network(X_state, name):
    prev_layer = X_state
    with tf.compat.v1.variable_scope(name) as scope:
        for n_maps, kernel_size, strides, padding, activation in zip(
            conv_n_maps, conv_kernel_sizes, conv_strides,
            conv_paddings, conv_activation):
            
            prev_layer = tf.compat.v1.layers.conv2d(
                prev_layer, filters=n_maps, kernel_size= kernel_size,
                strides= strides, padding=padding, activation=activation,
                kernel_initializer= initializer)
            
            last_conv_layer_flat = tf.reshape(prev_layer, shape=[-1, n_hidden_in])
            
            hidden = tf.compat.v1.layers.dense(last_conv_layer_flat, n_hidden, 
                                                activation=hidden_activation,
                                                kernel_initializer=initializer)
            
            outputs = tf.compat.v1.layers.dense(hidden, n_outputs, 
                                                kernel_initializer=initializer)
        
        trainable_vars = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, 
                                                    scope=scope.name)
        
        trainable_vars_by_name = {var.name[len(scope.name):] : var for var in trainable_vars}

        return outputs, trainable_vars_by_name
            


In [25]:
X_state = tf.compat.v1.placeholder(tf.float32, shape=[None, input_height, input_width, input_channels])
online_q_values, online_vars = q_network(X_state=X_state, name='q_networks/online')
target_q_values, target_vars = q_network(X_state=X_state, name='q_networks/target')

copy_obs = [target_var.assign(online_vars[var_name]) for var_name, target_var in target_vars.items()]
copy_online_to_target = tf.group(*copy_obs)

/opt/anaconda3/envs/DA_Enviroment/lib/python3.9/site-packages/tensorflow/python/keras/legacy_tf_layers/convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
/opt/anaconda3/envs/DA_Enviroment/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/opt/anaconda3/envs/DA_Enviroment/lib/python3.9/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '


In [9]:
# discount rewards function

def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.empty(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

In [10]:
# check function

discount_rewards([10,0,50], 0.8)

array([42., 40., 50.])

In [11]:
# discount and normalize rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    
    pass

In [12]:
res = env.render(mode='rgb_array')

In [18]:
env.render(mode='human')

AttributeError: 'NoneType' object has no attribute 'fill'

In [17]:

env.step([0,1,0])

AttributeError: 'Space_Docking_Env' object has no attribute 'player'